### Useful structures

In [98]:
class Node:
    idx = -1
    def __init__(self, c, parent=None, depth=0):
        self.c = c
        self.parent = parent
        self.depth = depth
        self.children = {}
        self.link = None
        self.idx = self.next_idx()
    
    def child(self, c):
        """ find child with label c """
        return self.children.get(c, None)
        
    def add_child(self, c):
        if c in self.children.keys():
            print("child already exists")
            return self.children[c]
        self.children[c] = Node(c, self, self.depth+1)
        return self.children[c]
    
    def find(self, word: list):
        """ find word in a trie of which self is root """
        if len(word) == 0:
            return True
        if self.child(word[0]) is None:
            return False
        return self.child(word[0]).find(word[1:])
    
    def next_idx(self):  
        """ finding unique idx for each node (idx of last-added node = size of trie-1)
        (unless several tries created at once)"""
        if not self.parent:
            Node.idx = -1
        Node.idx += 1
        return Node.idx
    
    def __repr__(self):
        return f"{self.c}-{self.idx}"

### Suffix Trie implementation

In [121]:
from queue import LifoQueue as queue 

def up_link_down(sibling):
    letters = queue()
    while(sibling and not sibling.link):
        letters.put(sibling.c)
        sibling=sibling.parent
    if(not sibling):
        return(None,None)
    node=sibling.link
    current_letter=letters.get()
    while(current_letter):
        if(node.child(current_letter)):
            node=node.child(current_letter)
            sibling=sibling.child(current_letter)
            sibling.link = node
        else:
            break
        current_letter=letters.get()
    return(node,sibling)

def graft(node, fragment, sibling=None):
#     print(f"fragment length {len(fragment)}, {fragment}")
    for letter in list(fragment):
        node = node.add_child(letter)
        if(sibling):
            sibling=sibling.child(letter)
            sibling.link = node
    return node

""" building trie """
def left_to_right(text):
    root = Node("")
    leaf = graft(root, text)
    root.child(text[0]).link = root
    for i in range(1, len(text)):
        head, sibling = up_link_down(leaf)
        if not head:
            sibling = root.child(text[i-1]) # redundant?
            sibling.link = root
            head = root
        leaf = graft(head,text[i+head.depth:],sibling)
        root.child(text[i]).link = root # experimental
    return root


In [122]:
def check(trie, text, w_print=False):
    """ check if trie contains all suffixes from text"""
    for i in range(len(text)):
        if not trie.find(text[i:]):
            if w_print:
                print(f"suffix {text[i:]} not found")
            return False
    if w_print:
        print("trie correct")
    return True

In [123]:
text = "bbbd"
trie = left_to_right(text)
check(trie, text, True)

trie correct


True

In [124]:
text = "aabbabd"
trie = left_to_right(text)
check(trie, text, True)

trie correct


True

In [125]:
text = "ababcd"
trie = left_to_right(text)
check(trie, text, True)

trie correct


True

In [69]:
text = "abcbccd"
trie = left_to_right(text)
check(trie, text, True)

fragment length 7, abcbccd
fragment length 6, bcbccd
fragment length 5, cbccd
fragment length 4, bccd
child already exists
child already exists
fragment length 2, cd
fragment length 1, d
fragment length 1, d


In [70]:
# file
text = "bbbd"
trie = left_to_right(text)
check(trie, text, True)

fragment length 4, bbbd
fragment length 1, d
fragment length 1, d
fragment length 1, d


In [126]:
text = "bbbaaaaababaabbd"
trie = left_to_right(text)
check(trie, text, True)

trie correct


True